# k-Means
Der k-Means-Algorithmus ist eines der beliebtesten Verfahren zur Clusteranalyse. Es lässt sich sehr einfach implementieren und findet schnell entsprechende Clusterzentren.

In [ ]:
import numpy as np

from tui_dsmt.clustering import kmeans, animate_kmeans, interactive_kmeans
from tui_dsmt.clustering.datasets import clustering_example1, clustering_example2

## Inhaltsverzeichnis
- [Clustering durch Varianzminimierung](#Clustering-durch-Varianzminimierung)
  - [Zentroide](#Zentroide)
  - [Kompaktheit](#Kompaktheit)
  - [Algorithmus](#Algorithmus)
  - [Verbesserte Initialisierung](#Verbesserte-Initialisierung)
- [k-Means](#k-Means)
- [Erweiterte Implementierungen](#Erweiterte-Implementierungen)
- [Probleme](#Probleme)

## Clustering durch Varianzminimierung

### Zentroide
k-Means verwendet sogenannte Zentroide, also künstliche Punkte, die als Repräsentanten für jeweils einen Cluster gelten. Das Ziel ist, eine feste Anzahl dieser Zentroide genau so zu verteilen, dass die Varianz innerhalb der Cluster möglichst gering wird. Es ist also ein Abstandsmaß $dist$ notwendig, wobei im Folgenden die euklidische Distanz zur Anwendung kommt.

### Kompaktheit
Zur Durchführung wird ein Qualitätskriterium benötigt, das im Verlaufe des Algorithmus optimiert wird. Die **Kompaktheit** beim Clustering durch Varianzminimierung und auch beim k-Means-Algorithmus beschreibt, wie eng die Datenpunkte innerhalb jedes Clusters um dessen Mittelpunkt (Zentroid) liegen. Sie wird durch die Minimierung der Summe der quadratischen Abstände aller Punkte zu ihrem jeweiligen Zentroid erreicht, was eine möglichst dichte Gruppierung der Punkte sicherstellt.

Die Kompaktheit $TD^2$ für einen spezifischen Cluster $C$ kann wie folgt formuliert werden, wobei $\overline{x}_C$ den Zentroiden des Clusters bezeichnet:
$$
TD^2(C) = \sum_{p \in C} dist(p, \overline{x}_C)^2
$$

Die Kompaktheit eines Clusterings definiert sich dann als Summe über die Kompaktheit aller entstandener Cluster:
$$
TD^2 = \sum_{i=1}^{k} TD^2(C_i)
$$

### Algorithmus
k-Means basiert auf einem Algorithmus, der als *Clustering durch Varianzminimierung* bezeichnet wird. Der Algorithmus verläuft wie folgt:
1. Ordne alle Punkte zufällig einem der $k$ Cluster zu. Die Anzahl $k$ muss also vorher bekannt sein. (Cluster dürfen nicht leer sein!)
2. wiederhole bis zur Konvergenz:
    1. Jeder Punkt wird dem Cluster zugeordnet, dessen Zentrum er am nächsten liegt.
    2. Jedes Clusterzentrum wird als Mittelpunkt aller zugehörigen Punkte neu bestimmt.

Zuerst implementieren wir eine Hilfsfunktion, die Zentroide auf Basis der ihnen zugeordneten Punkte berechnet.

In [ ]:
def calculate_centroids(xs, ys, clusters, k):
    # Alle Punkte und ihre jeweilige Clusterzuordnung
    # werden verwendet, um eine Summe über die x- und
    # y-Koordinaten zu bilden und die Anzahl der Punkte
    # pro Cluster zu bestimmen.
    centroids_x_sum = np.zeros(k)
    centroids_y_sum = np.zeros(k)
    centroids_len = np.zeros(k)

    for x, y, c in zip(xs, ys, clusters):
        centroids_x_sum[c] += x
        centroids_y_sum[c] += y
        centroids_len[c] += 1

    # Clustermittelpunkte, denen keine Punkte zugeordnet
    # wurden, werden zufällig verteilt, um leere Cluster
    # zu vermeiden.
    for i in range(len(centroids_len)):
        if centroids_len[i] == 0:
            random_index = np.random.randint(0, len(xs))
            centroids_x_sum[i] = xs[random_index]
            centroids_y_sum[i] = ys[random_index]
            centroids_len[i] = 1

    # Der Durchschschnitt der Punkte pro Cluster bildet
    # die Zentroide.
    return centroids_x_sum / centroids_len, centroids_y_sum / centroids_len

Die folgende Funktion arbeitet als Generator und gibt immer wieder neue Clusterzentren zurück, bis sie nicht mehr aufgerufen wird. Die Prüfung des Abbruchkriteriums fällt damit in die Verantwortung des aufrufenden Programmbestandteils.

In [ ]:
def cdvm(xs, ys, k):
    # Allen Punkten wird zufällig ein Cluster zugeordnet.
    clusters = np.concatenate((
        np.arange(0, k),
        np.random.randint(0, k, size=len(xs) - k)
    ))
    np.random.shuffle(clusters)

    # Die Clusterzentren der aktuellen Konfiguration werden bestimmt.
    centroids_x, centroids_y = calculate_centroids(xs, ys, clusters, k)
    yield centroids_x, centroids_y

    # Die Clusterzentren werden wiederholt neu bestimmt.
    while True:
        # Die Clusterzugehörigkeit wird für alle Punkte neu bestimmt.
        for i, (x, y) in enumerate(zip(xs, ys)):
            # Dazu wird die Distanz zu jedem Clustermittelpunkt berechnet.
            distances = np.sqrt((centroids_x - x) ** 2 + (centroids_y - y) ** 2)

            # Der Cluster des Punktes wird auf den nächstgelegenen
            # Zentroiden gesetzt.
            closest_centroid_index = distances.argmin()
            clusters[i] = closest_centroid_index

        # Die Zentroide werden neu bestimmt.
        centroids_x, centroids_y = calculate_centroids(xs, ys, clusters, k)
        yield centroids_x, centroids_y

Der implementierte Algorithmus kann nun auf den Datensatz angewendet werden. Clustering durch Varianzminimierung ist stark von der zufälligen Initialisierung abhängig. Führen Sie die Zelle mehrfach aus und beobachten Sie die Entwicklung basierend auf den Zentroiden im ersten Frame.

In [ ]:
animate_kmeans(cdvm, clustering_example1, k=15)

### Verbesserte Initialisierung
Die Zentroide befinden sich auf Grund der zufälligen Zuweisung der Punkte und Bilden des Durchschnitts zu Beginn des Algorithmus überwiegend in der *Mitte* des Datensatzes und bleiben gegebenenfalls im weiteren Verlauf auch dort "gefangen". Eine einfache Alternative zur Initialisierung besteht in der Zuordnung der Zentroide zu einzelnen Punkten. Dadurch ist die Wahrscheinlichkeit höher, dass einzelne Zentroide zum Start des Algorithmus in äußere Bereichen gelegt werden.

In [ ]:
def cdvm2(xs, ys, k):
    # Die Zentroide werden zufällig aus der Menge der
    # Punkte ausgewählt.
    random_indices = np.random.randint(0, len(xs), k)
    centroids_x, centroids_y = xs[random_indices], ys[random_indices]
    yield centroids_x, centroids_y

    # Die Clusterzentren werden wiederholt neu bestimmt.
    clusters = np.zeros(len(xs), dtype=int)

    while True:
        # Die Clusterzugehörigkeit wird für alle Punkte neu bestimmt.
        for i, (x, y) in enumerate(zip(xs, ys)):
            # Dazu wird die Distanz zu jedem Clustermittelpunkt berechnet.
            distances = np.sqrt((centroids_x - x) ** 2 + (centroids_y - y) ** 2)

            # Der Cluster des Punktes wird auf den nächstgelegenen
            # Zentroiden gesetzt.
            closest_centroid_index = distances.argmin()
            clusters[i] = closest_centroid_index

        # Die Zentroide werden neu bestimmt.
        centroids_x, centroids_y = calculate_centroids(xs, ys, clusters, k)
        yield centroids_x, centroids_y

Führen Sie auch diesen Algorithmus mehrfach aus und beobachten Sie die Auswirkungen der geänderten Initialisierung.

In [ ]:
animate_kmeans(cdvm2, clustering_example1, k=15)

## k-Means
k-Means ist eine Variante des Basisalgorithmus, bei der die Zentroide jedes Mal direkt aktualisiert werden, wenn ein Punkt in einen anderen Cluster wechselt. Um nicht jedes Mal alle Cluster neu zu berechnen, werden nur die beiden Zentroide des Clusters $C_1$, der einen Punkt abgibt, und $C_2$, der einen Punkt hinzugewinnt, aktualisiert. Die neuen Zentroide der veränderten Cluster $C_1^,$ und $C_2^,$ ergeben sich dann wie folgt:

$$
\overline{x}_j(C_1^,) = \frac{1}{n_{C_1}-1} * (n_{C_1} * \overline{x}_j(C_1) - x_j^p)
$$

$$
\overline{x}_j(C_2^,) = \frac{1}{n_{C_2}+1} * (n_{C_2} * \overline{x}_j(C_2) + x_j^p)
$$

$n_{C_i}$ bezeichnet dabei die Anzahl der Punkte, die sich im Cluster $C_i$ befindet.

Die Anpassung kann zum Beispiel wie folgt in unseren Basisalgorithmus eingebracht werden.

In [ ]:
def kmeans(xs, ys, k):
    # Allen Punkten wird zufällig ein Cluster zugeordnet.
    clusters = np.concatenate((
        np.arange(0, k),
        np.random.randint(0, k, size=len(xs) - k)
    ))
    np.random.shuffle(clusters)

    # Die Clusterzentren der aktuellen Konfiguration werden bestimmt.
    centroids_x, centroids_y = calculate_centroids(xs, ys, clusters, k)
    yield centroids_x, centroids_y

    # Die Clusterzentren werden wiederholt neu bestimmt.
    while True:
        # Die Clusterzugehörigkeit wird für alle Punkte neu bestimmt.
        for i, (x, y) in enumerate(zip(xs, ys)):
            # Dazu wird die Distanz zu jedem Clustermittelpunkt berechnet.
            distances = np.sqrt((centroids_x - x) ** 2 + (centroids_y - y) ** 2)

            old_cluster = clusters[i]
            new_cluster = distances.argmin()

            # Wenn sich die Clusterzuordnung eines Punktes verändert,
            # werden die Zentroide sofort neu bestimmt.
            if old_cluster != new_cluster:
                n_old_cluster = np.sum(clusters == old_cluster)
                centroids_x[old_cluster] = 1 / (n_old_cluster - 1) * (n_old_cluster * centroids_x[old_cluster] - x)
                centroids_y[old_cluster] = 1 / (n_old_cluster - 1) * (n_old_cluster * centroids_y[old_cluster] - y)

                n_new_cluster = np.sum(clusters == new_cluster)
                centroids_x[new_cluster] = 1 / (n_new_cluster + 1) * (n_new_cluster * centroids_x[new_cluster] + x)
                centroids_y[new_cluster] = 1 / (n_new_cluster + 1) * (n_new_cluster * centroids_y[new_cluster] + y)

                clusters[i] = new_cluster

        # Die Zentroide werden erst nach der Iteration über alle Punkte
        # zurückgegeben, um nicht zu viele Bilder zu erzeugen.
        yield centroids_x, centroids_y

Auch diesen Algorithmus können Sie bei der Arbeit betrachten.

In [ ]:
animate_kmeans(kmeans, clustering_example1, k=15)

## Erweiterte Implementierungen
Wie Sie gesehen haben, hängt der Algorithmus von der Initialisierung des Zufallszahlengenerators ab. Sie können daher beobachten, dass einige der Häufungen im äußeren Bereich aufgeteilt werden, während sich übergreifende Cluster in der Mitte bilden. Es kann sich daher lohnen, die Methode mehrfach auszuführen und das beste Ergebnis auszuwählen. Eine fertige Implementierung, die den k-Means-Algorithmus mehrfach ausführt und weitere Verbesserungen enthält, entstammt dem Paket `sklearn`.

Initialisieren Sie dazu ein Objekt der Klasse `KMeans` und übergeben Sie dem Parameter die Anzahl der Clusterzentren. Rufen Sie anschließend `fit_predict` mit einem DataFrame auf, das die Angaben zu den entsprechenden Dimensionen enthält. Zurückgegeben wird ein Array aus Clusterindizes.

In [ ]:
from sklearn.cluster import KMeans

k = 15
KMeans(k).fit_predict(clustering_example1[['x', 'y']])

In der nachfolgenden Zelle haben Sie die Möglichkeit mit dem Parameter $k$ zu experimentieren.

In [ ]:
interactive_kmeans(clustering_example1)

## Probleme
Der k-Means Algorithmus eignet sich jedoch nicht für alle Daten. Mengen, die nicht konvex sind, können beispielsweise nicht korrekt in Cluster eingeteilt werden. Die ineinandergreifenden Spiralen demonstrieren anschaulich die Grenzen des Algorithmus.

In [ ]:
animate_kmeans(kmeans, clustering_example2, k=3)